<a href="https://colab.research.google.com/github/anniedendas/NUWIT-website-hack-day-2022/blob/main/PageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PageRank

In this exercise, you will compute PageRank on a collection of 469,235 web sites using the iterative version of the PageRank algorithm described in class for sparse graphs.  As we discussed in class, this avoids performing cubic-time operations on a graph with a large number of nodes.

Furthermore, you should take advantage of the _sparsity_ of the original hyperlinks, even though "teleportation" or random jumps connect every note to every other node.  As long as the maximum degree of the original link graph is much less than the number of nodes, you should be able to keep each iteration's runtime less than quadratic in the number of nodes.

Consider the following directed graph:

![A directed link graph](https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200//pagerank.jpg)

We can represent this graph as a collection of nodes, here, ordered pairs of node index and node name:

In [1]:
small_nodes = [(0, 'A'),
              (1, 'B'),
              (2, 'C'),
              (3, 'D'),
              (4, 'E'),
              (5, 'F')]

and a collection of directed links, i.e., ordered pairs from source to target:

In [2]:
small_edges = [
  (0, 1),
  (0, 2),
  (0, 5),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (2, 3),
  (2, 4),
  (3, 0),
  (3, 2),
  (3, 4),
  (3, 5),
  (4, 0),
  (5, 0),
  (5, 1),
  (5, 4)
]

We use integer identifiers for the nodes for efficiency. In most real-world collections of hyperlinks, unlike this example, not every page will have in-links, nor will every page have out-links.

## First Implementation and Test

\[10 points\] Implement the iterative PageRank algorithm we discussed in class. Test your code on the six-node example using the input representation given above.  Be sure that your code handles pages that have no in-links or out-links properly.  (You may wish to test on a few such examples.) In later parts of this assignment, depending on how you store the data, it may be convenient to use iterators rather than storing the data in memory.

In [3]:
# TODO: Implement PageRank, given nodes and edges, to start with a uniform
# distribution over nodes, run a fixed number of iterations, and
# return a distribution over nodes.

def page_rank_fixed_iter(nodes, edges, iterations=10, d=0.1):
    n = len(nodes)

    # initialize PageRank values
    I = {node[0]: 1.0 / n for node in nodes}  # current PageRank estimate
    R = {node[0]: 0.0 for node in nodes}      # updated PageRank estimate

    # create adjacency list and outlink counts
    adj_list = {node[0]: [] for node in nodes}
    outlink_counts = {node[0]: 0 for node in nodes}

    # populate adjacency list and outlink counts
    for source, target in edges:
        adj_list[source].append(target)
        outlink_counts[source] += 1

    for _ in range(iterations):
        # set the base value for each node
        for node_id in range(n):
            p = nodes[node_id][0]
            R[p] = (1 - d) / n

            # get the set of pages linking to p
            Q = adj_list[p]

            if len(Q) > 0:
                # distribute PageRank to its outlink targets
                for q in Q:
                    R[q] += d * (I[p] / outlink_counts[p])
            else:
                # if no outlinks, distribute PageRank equally
                for q in range(n):
                    R[nodes[q][0]] += d * (I[p] / n)

        # update the PageRank estimates for the next iteration
        I = R.copy()

    # return final PageRank values as a list of tuples
    return [(node[0], node[1], R[node[0]]) for node in nodes]



# Output PageRank on the toy graph at various points.
# Make sure your output has node number, name, and PageRank value.
page_rank_fixed_iter(small_nodes, small_edges, 1)
page_rank_fixed_iter(small_nodes, small_edges, 10)
page_rank_fixed_iter(small_nodes, small_edges, 100)

[(0, 'A', 0.17425000000000002),
 (1, 'B', 0.155),
 (2, 'C', 0.15375),
 (3, 'D', 0.15),
 (4, 'E', 0.155),
 (5, 'F', 0.15)]

## PageRank on Web Crawl Data

\[20 points\] Download and unpack a list of `.edu` websites and the links among them from the [Common Crawl](https://commoncrawl.org/2017/05/hostgraph-2017-feb-mar-apr-crawls/) open-source web crawl. For the sake of brevity, the data record links among websites, not web pages. The information for nodes and links is the same as the toy example above.

In [4]:
# If you're running on a machine (e.g., Windows) that doesn't have wget or gzip,
# feel free to comment this out and use a different set of commands to load
# the data.
!wget https://khoury.northeastern.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
!gzip -df vertices-edu.txt.gz
!wget https://khoury.northeastern.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
!gzip -df edges-edu.txt.gz

--2024-10-16 20:21:04--  https://khoury.northeastern.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
Resolving khoury.northeastern.edu (khoury.northeastern.edu)... 52.70.229.197
Connecting to khoury.northeastern.edu (khoury.northeastern.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz [following]
--2024-10-16 20:21:04--  https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
Resolving www.khoury.northeastern.edu (www.khoury.northeastern.edu)... 52.70.229.197
Connecting to www.khoury.northeastern.edu (www.khoury.northeastern.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3703486 (3.5M) [application/x-gzip]
Saving to: ‘vertices-edu.txt.gz’

vertices-edu.txt.gz 100%[===================>]   3.53M  8.10MB/s    in 0.4s    

2024-10-16 20:21:05 (8.10 MB/s) - ‘vertices-edu.

There should now be files `vertices-edu.txt` and `edges-edu.txt`.

In [5]:
# TODO: Process the raw data into the same format as the simple graph.
# You may create lists or iterators.

# process vertices file to create the list of nodes
nodes = []
with open('vertices-edu.txt', 'r') as f:
    for line in f:
        parts = line.strip().split()  # split the line into components
        node_id = int(parts[0])       # node ID
        website_name = parts[1]       # website name
        nodes.append((node_id, website_name))  # append as a tuple

# process edges file to create the list of edges
edges = []
with open('edges-edu.txt', 'r') as f:
    for line in f:
        parts = line.strip().split()  # split the line into components
        source_node_id = int(parts[0])  # source node ID
        target_node_id = int(parts[1])  # target node ID
        edges.append((source_node_id, target_node_id))

# check that nodes and edges are correct
print({len(nodes)})
print({len(edges)})


{469235}
{3300462}


Refine your implementation of PageRank to test for numerical convergence. Specificially, at each iteration, calculate the [perplexity](https://en.wikipedia.org/wiki/Perplexity) of the PageRank distribution, where perplexity is defined as 2 raised to the [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) of the PageRank distribution, i.e., $2^{H(PR)}$. (Recall the definition of entropy from our discussion of data compression.) The maximum perplexity of a PageRank distribution will therefore be the number of nodes in the graph.

At each iteration, check the _change_ in perplexity. If the change is less than some threshold, you can stop.


In [6]:
# TODO: Implement convergence testing in PageRank
# If you choose, you can share some subroutines with your first version.
# Print the change in perplexity at each iteration.
import math

# calculate the perplexity of the PageRank dist
def find_perplexity(I):
    entropy = 0
    for rank in I.values():
        if rank > 0:
            entropy += rank * math.log2(rank)
    entropy = -entropy
    return 2 ** entropy

def page_rank(nodes, edges, threshold=1, d=0.1, max_iterations=100):
    n = len(nodes)

    # initialize PageRank values
    I = {node[0]: 1.0 / n for node in nodes}  # current PageRank estimate
    R = {node[0]: 0.0 for node in nodes}      # updated PageRank estimate

    # create adjacency list and outlink counts
    adj_list = {node[0]: [] for node in nodes}
    outlink_counts = {node[0]: 0 for node in nodes}

    # populate adjacency list and outlink counts
    for source, target in edges:
        adj_list[source].append(target)
        outlink_counts[source] += 1

    previous_perplexity = find_perplexity(I)
    print(f"Initial perplexity: {previous_perplexity}")

    for iteration in range(max_iterations):
        discon_sum = sum(I[p] for p in I if outlink_counts[p] == 0)  # sum of PageRank from disconnected nodes

        # distribute the base value and disconnected node contribution
        for p in range(n):
            R[nodes[p][0]] = (1 - d) / n + d * (discon_sum / n)

        # distribute PageRank to outlink targets
        for p, outlinks in adj_list.items():
            if outlinks:
                share = d * (I[p] / outlink_counts[p])
                for q in outlinks:
                    R[q] += share

        # calculate new perplexity
        new_perplexity = find_perplexity(R)
        print(f"Iteration {iteration + 1}, Perplexity: {new_perplexity}, Change: {abs(new_perplexity - previous_perplexity)}")

        # check for convergence
        if abs(new_perplexity - previous_perplexity) < threshold:
            print(f"Convergence reached at iteration {iteration + 1}")
            break

        # update PageRank estimates for the next iteration
        I = R.copy()
        previous_perplexity = new_perplexity

    # return final PageRank values as a list of tuples
    return [(node[0], node[1], R[node[0]]) for node in nodes]

# Run until perplexity changes by less than 1
PR = page_rank(nodes, edges, 1)

Initial perplexity: 469234.99998989614
Iteration 1, Perplexity: 460906.9021881449, Change: 8328.097801751224
Iteration 2, Perplexity: 460711.5750317206, Change: 195.32715642428957
Iteration 3, Perplexity: 460695.99348618067, Change: 15.581545539957006
Iteration 4, Perplexity: 460695.4671955734, Change: 0.5262906072894111
Convergence reached at iteration 4


## Link Analysis

\[20 points\] In this final section, you will compute some properties of this web-site graph.

First, consider the _in-link count_ of a website, simply the number of websites pointing to it (including self-links).

In [7]:
# TODO: List the document ID, domain name, and in-link count of the 70 websites with the highest in-link count

# initialize inlink counts
inlink_counts = {node[0]: 0 for node in nodes}

# populate inlink counts
for source, target in edges:
    inlink_counts[target] += 1

# create a list containing document ID, domain name, inlink count as tuples
inlink_data = [(node[0], node[1], inlink_counts[node[0]]) for node in nodes]

# sort by inlink count in desc order and select the top 70
top_inlinks = sorted(inlink_data, key=lambda x: x[2], reverse=True)[:70]

# print the top 70 inlinks
for inlink in top_inlinks:
    print("Document ID | Domain Name | Inlink Count")
for doc_id, domain, count in top_inlinks:
    print(f"{doc_id: <12} | {domain: <20} | {count}")


Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Domain Name | Inlink Count
Document ID | Do

Then, use the PageRank values compute by your second implementation. Note that some websites will have both a high in-link count and PageRank.

In [ ]:
# TODO: List the document ID, domain name, and PageRank of the 70 websites with the highest PageRank.

Finally, compute some summary statistics on this dataset.

In [ ]:
# TODO: Compute:
# - the proportion of websites with no in-links (i.e., source nodes);

# - the proportion of websites with no out-links (i.e., sink nodes);

# - the proportion of websites whose PageRank is higher than the initial uniform distribution.
